Task:

Analyze demographic data which was extracted from the 1994 census database.

## Stage 1. Data acquisition

### Let's examine the data provided for this project

The data is from https://www.kaggle.com/datasets/uciml/adult-census-income

About Dataset

This data was extracted from the 1994 Census bureau database by Ronny Kohavi and Barry Becker (Data Mining and Visualization, Silicon Graphics). A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1) && (HRSWK>0)). The prediction task is to determine whether a person makes over $50K a year.

Description of fnlwgt (final weight)

The weights on the Current Population Survey (CPS) files are controlled to independent estimates of the civilian noninstitutional population of the US. These are prepared monthly for us by Population Division here at the Census Bureau. We use 3 sets of controls. These are:

    1 - A single cell estimate of the population 16+ for each state.

    2 - Controls for Hispanic Origin by age and sex.

    3 - Controls by Race, age and sex.

We use all three sets of controls in our weighting program and "rake" through them 6 times so that by the end we come back to all the controls we used. The term estimate refers to population totals derived from CPS by creating "weighted tallies" of any specified socio-economic characteristics of the population. People with similar demographic characteristics should have similar weights. There is one important caveat to remember about this statement. That is that since the CPS sample is actually a collection of 51 state samples, each with its own probability of selection, the statement only applies within state.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('adult.csv')
adult_data = data.copy() # make a deep copy to avoid bad manipulation of the main data

Let's analyze in detail which columns are in adult_data and what information they contain:

In [3]:
adult_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [4]:
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


So, there are 15 columns in the table: non-null int64 (6 columns) and non-null object (9 columns)

Let's check if there is missing values in our data

In [5]:
adult_data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
salary            0
dtype: int64

Our data does not contain missing values so we can analyze our data by anwering some questions

## Exploratory data analysis

##### Let's check how many people for each race are represented in our data

In [6]:
adult_data['race'].value_counts()

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

We can see that about 85% of the interviewed person are white

##### Let's check the average age of men

Filtering

In [9]:
sex_male = adult_data['sex']=='Male'
data_age_men = adult_data['age'].where(sex_male)
average_age_men = round(data_age_men.mean(),1)
print(f'{average_age_men} is the average age of men')

39.4 is the average age of men


##### Let's check the percentage of people who have bachelor's degree

Filtering

In [10]:
bachelors = adult_data['education'] == 'Bachelors'

In [11]:
# Get the number of people who have bachelor degree
number_bachelor = adult_data['education'].where(bachelors)

In [12]:
# We can get the percentage by dividing this latter by the total number multiplied by 100
percentage_bachelor = round((number_bachelor.count()*100)/adult_data['education'].count(),2)

In [13]:
percentage_bachelor

16.45

Only 16.45% have bachelor's degree

##### Let's see the percentage of people with advanced education (Bachelors, Masters or Doctorate) who make more than 50K

In [16]:
# person with bachelor or master or doctorate degree
advance_education = (adult_data['education']=='Bachelors') | (adult_data['education']=='Masters') | (adult_data['education'] == 'Doctorate')            

In [17]:
# Salary great than 50K
salary_great_50K = adult_data['salary'] == '>50K'

In [18]:
# number of person with high degree
high_education = adult_data['education'].where(advance_education).count()

In [19]:
# percentage of person with high degree and make more than 50K
# which is the number of person with high degree and who make more than 50K divided by the total number of 
# with high degree multiplied by 100
percentage_high_education = round(adult_data['education'].where(advance_education & salary_great_50K).count() * 100/
                                  high_education,1)
print(f'There is {percentage_high_education}% of people with advanced degree who make more than 50K in the dataset')

There is 46.5% of people with advanced degree who make more than 50K in the dataset


##### Let's see the percentage of people without advanced degree but make more than 50K

In [22]:
# person without bachelor or master or doctorate degree
not_advance_education = ~(adult_data['education']=='Bachelors') & ~(adult_data['education']=='Masters') & ~(adult_data['education'] == 'Doctorate')   

In [23]:
# number of person without advanced education
low_education = adult_data['education'].where(not_advance_education).count()

In [24]:
percentage_low_education = round(adult_data['education'].where(not_advance_education & low_education).count() * 100 
                                 / low_education,1)
print(f'There is {percentage_low_education}% of people without advanced degree who make more than 50K in the dataset')

There is 0.0% of people without advanced degree who make more than 50K in the dataset


##### Let's see the minimum number of hours a person work per week

In [25]:
min_work_hours = adult_data['hours-per-week'].min()
print(f'The minimum number of hours a person wotk per week is {min_work_hours}h')

The minimum number of hours a person wotk per week is 1h


##### Let's see what percentage of people who work the minimum number of hours per week have a salary more than 50K

In [26]:
# number of person who work the minimum number of hours per week and have salary more than 50K
number_pers_min_work_h = len(adult_data[(adult_data['hours-per-week']==min_work_hours) & salary_great_50K])

In [27]:
# number of person who work the minimum number of hours per week
number_pers_work_min_hour = len(adult_data[adult_data['hours-per-week']==min_work_hours])

In [28]:
# percentage of people who work the minimum number of hours per week have a salary more than 50K
percentage_pers_work_min_hour = (number_pers_min_work_h*100)/number_pers_work_min_hour
print(f'{percentage_pers_work_min_hour}% of the interviewed person work {min_work_hours}h per week and make more than 50K')

10.0% of the interviewed person work 1h per week and make more than 50K


##### Let's see which country has the highest percentage of people that earn more than 50K

In [29]:
# create new dataframe with all the country in its entries
country_list = pd.DataFrame(data=adult_data['native-country'].unique(), columns = ["native-country"])

In [30]:
# convert it to be the index of the dataframe
country_list.set_index('native-country')

""
native-country
United-States
Cuba
Jamaica
India
?
Mexico
South
Puerto-Rico
Honduras


In [31]:
country_list['number_of_earners'] = 0
country_list['percentage'] = 0

In [33]:
# filtering
for i, row in country_list.iterrows():
    country = adult_data['native-country'] == row['native-country']
    country_list.iloc[i, 1] = adult_data['native-country'].where(salary_great_50K & country).count()
    country_list.iloc[i, 2] = country_list.iloc[i, 1] / adult_data['native-country'].where(country).count() * 100

In [34]:
highest_country = country_list.loc[country_list['percentage'].idxmax(), 'native-country']
percentage_highest_country = round(country_list.loc[country_list['percentage'].idxmax(), 'percentage'],1)
print(f'{highest_country} is the country that has highest percentage of people that earn more than 50K')
print(f'That percentage is {percentage_highest_country}%')

Iran is the country that has highest percentage of people that earn more than 50K
That percentage is 41.9%


##### Let's see the most popular occupation for those who earn more than 50K in India

In [35]:
country_India = adult_data['native-country']=='India'
India_occupation = adult_data[(country_India) & (salary_great_50K)]
popular_occupation = India_occupation['occupation'].value_counts().idxmax()
print(f'The most most popular occupation for those who earn more than 50K in India is {popular_occupation}')

The most most popular occupation for those who earn more than 50K in India is Prof-specialty






That is for this project, I had the opportunity to explore through the data by answering different questions.

You can still explore the data by answering other questions if you want to.